In [1]:
# Python lib
import os
import pandas as pd

# External lib
import plotly.express as px
from splink.duckdb.duckdb_linker import DuckDBLinker
from splink.duckdb.duckdb_comparison_library import (
    exact_match,
    levenshtein_at_thresholds,
)

# Local lib
import toolkit as tk
import geovpylib as gv

# Connect to db
tk.db_connect(os.environ.get('GEOVISTORY_STAGING_URL'))

>> Connecting to PGSQL Database ... Connected!


# Fetch data from BHP (from cleaned filed)

In [2]:
actor = pd.read_csv('../../data/actor.csv', sep=';')

tk.set_types(actor, {
               "pk_actor": 'int', 
      "modification_time": 'datetime',
          "creation_time": 'datetime',
   "concat_standard_name": 'string',
            "concat_actr": 'string',
                "creator": 'int',
             "gender_iso": 'int',
               "modifier": 'int',
        "certainty_begin": 'int',
          "certainty_end": 'int',
             "begin_year": 'int',
               "end_year": 'int',
            "notes_begin": 'int',
              "notes_end": 'int',
                  "notes": 'string'
})

In [3]:
actor_name = pd.read_csv('../../data/actor_name.csv', sep=';', low_memory=False)

tk.set_types(actor_name, {
     "pk_actor_name": 'int',
       "concat_acna": 'string',
     "creation_time": 'datetime',
  "is_standard_name": 'bool',
          "fk_actor": 'int',
       "concat_name": 'string',
           "creator": 'int',
              "name": 'string',
          "lang_iso": 'string',
          "modifier": 'int',
        "first_name": 'string',
 "modification_time": 'datetime',
 "fk_abob_name_type": 'int',
             "notes": 'string',
"comment_begin_year": 'string',
  "comment_end_year": 'string',
        "apposition": 'string',
       "preposition": 'string',
          "particle": 'string',
             "title": 'string',
        "begin_date": 'string',
          "end_date": 'string',
      "ordinal_text": 'string',
       "ordinal_num": 'int',
})

In [4]:
actor_text_property = pd.read_csv('../../data/actor_text_property.csv', sep=';', low_memory=False)

tk.set_types(actor_text_property, {
'pk_actor_text_property': 'int',
         'property_type': 'string',
                  'text': 'text',
              'fk_actor': 'int',
           'concat_actp': 'string',
         'creation_time': 'datetime',
               'creator': 'int',
         'lang_iso_code': 'string',
              'modifier': 'int',
     'modification_time': 'datetime',
                 'notes': 'string',
})     

## Create Person table

In [5]:
# Names
names = pd.DataFrame()
names['id'] = actor_name['fk_actor']
names['name'] = actor_name['first_name'] + ' ' + actor_name['name']
names['name'] = names['name'].str.lower()
names['name'] = names['name'].str.strip()
names.dropna(inplace=True)

print('Number of names available:', len(names))

Number of names available: 59711


In [6]:
def set_gender(code):
    """Given the ISO code of a gender, return the right string."""

    if code == 0: return pd.NA
    if code == 1: return 'Male'
    if code == 2: return 'Female'
    raise ValueError(f'Gender has code <{code}>')


# Gender
genders = pd.DataFrame()
genders['id'] = actor['pk_actor']
genders['gender'] = [set_gender(code) for code in actor['gender_iso']]
genders.dropna(inplace=True)

print('Number of genders available:', len(genders))

Number of genders available: 57753


In [7]:
# Births
births = pd.DataFrame()
births['id'] = actor['pk_actor']
births['birth_year'] = actor['begin_year']
births.dropna(inplace=True)

print('Number of births available:', len(births))

Number of births available: 48476


In [8]:
# Deaths
deaths = pd.DataFrame()
deaths['id'] = actor['pk_actor']
deaths['death_year'] = actor['end_year']
deaths.dropna(inplace=True)

print('Number of deaths available:', len(deaths))

Number of deaths available: 29366


In [9]:
# Merging all together

persons_bhp = pd.DataFrame()
persons_bhp['id'] = actor['pk_actor']
persons_bhp = persons_bhp.merge(names, on='id', how='left')
persons_bhp = persons_bhp.merge(genders, on='id', how='left')
persons_bhp = persons_bhp.merge(births, on='id', how='left')
persons_bhp = persons_bhp.merge(deaths, on='id', how='left')
persons_bhp['dataset'] = 'bhp'
persons_bhp.dropna(subset=['name', 'gender', 'birth_year', 'death_year'], how='all', inplace=True)
persons_bhp.drop_duplicates(inplace=True)

tk.infos(persons_bhp, random=True)

Shape:  (62528, 6)


,id,name,gender,birth_year,death_year,dataset
62218,43993,<NA>,Male,1723,<NA>,bhp
57710,62078,jules-louis mabille,Male,1843,1897,bhp
29931,28779,mme claudot,Female,1850,<NA>,bhp
11152,2408,felipe pascual,Male,1670,<NA>,bhp
52105,44098,jean berthenet,Male,1900,<NA>,bhp


In [10]:
persons_bhp.to_csv('../../data/persons-bhp.csv', index=False, sep=";", quoting=2)

# Fetch Data from Geovistory

In [11]:
print('Total persons number: ', tk.db_execute('select count(*) from information.resource where fk_class = 21;').iloc[0]['count'])
print('Total PAIAL number: ', tk.db_execute('select count(*) from information.resource where fk_class = 868;').iloc[0]['count'])
print('Total Births number: ', tk.db_execute('select count(*) from information.resource where fk_class = 61;').iloc[0]['count'])
print('Total Deaths number: ', tk.db_execute('select count(*) from information.resource where fk_class = 63;').iloc[0]['count'])

Total persons number:  118946
Total PAIAL number:  179649
Total Births number:  71830
Total Deaths number:  1430


In [12]:
persons_geov = tk.db_execute('select pk_entity as id from information.resource where fk_class = 21;')

print('Number of persons available:', len(persons_geov))

Number of persons available: 118946


In [13]:
# Names
names = tk.db_execute('''
    select 
        r1.pk_entity as id,
        a1.string as name
    from information.resource r1
    left join information.statement s1 on s1.fk_object_info = r1.pk_entity and s1.fk_property = 1111
    left join information.statement s2 on s2.fk_subject_info = s1.fk_subject_info and s2.fk_property = 1113
    left join information.appellation a1 on a1.pk_entity = s2.fk_object_info
    where r1.fk_class = 21;
''')
names.dropna(inplace=True)
names['name'] = names['name'].str.lower()
names['name'] = names['name'].str.strip()

print('Number of names available:', len(names))

Number of names available: 149924


In [14]:
# Genders
genders = tk.db_execute('''
    select distinct
        fk_subject_info as id,
        fk_object_info as gender
    from information."statement" s 
    inner join projects.info_proj_rel ipr on ipr.fk_entity = s.pk_entity and ipr.is_in_project = True
    where s.fk_property = 1429 and (s.fk_object_info = 739340 or s.fk_object_info = 739346);
''')
genders['gender'].replace(739340, 'Male', inplace=True)
genders['gender'].replace(739346, 'Female', inplace=True)
genders.dropna(inplace=True)

print('Number of genders available:', len(names))

Number of genders available: 149924


In [15]:
# Births
births = tk.db_execute('''
    select
        s1.fk_object_info as id,
        tp.julian_day as birth_date
    from information.statement s1
    inner join information.statement s2 on s2.fk_subject_info = s1.fk_subject_info and s2.fk_property = 72
    inner join information.time_primitive tp on tp.pk_entity = s2.fk_object_info
    where s1.fk_property = 86
''')
births['birth_date'] = [gv.tools.get_date_from_julian_day(jd)[0] if pd.notna(jd) else pd.NA for jd in births['birth_date']]
births.rename(columns={'birth_date': 'birth_year'}, inplace=True)
births.dropna(inplace=True)

print('Number of births available:', len(births))

Number of births available: 69459


In [16]:
# Deaths
deaths = tk.db_execute('''
    select
        s1.fk_object_info as id,
        tp.julian_day as death_date
    from information.statement s1
    inner join information.statement s2 on s2.fk_subject_info = s1.fk_subject_info and s2.fk_property = 72
    inner join information.time_primitive tp on tp.pk_entity = s2.fk_object_info
    where s1.fk_property = 88
''')
deaths['death_date'] = [gv.tools.get_date_from_julian_day(jd)[0] if pd.notna(jd) else pd.NA for jd in deaths['death_date']]
deaths.rename(columns={'death_date': 'death_year'}, inplace=True)
deaths.dropna(inplace=True)

print('Number of deaths available:', len(births))

Number of deaths available: 69459


In [17]:
# Merging all together

persons_geov = persons_geov.merge(names, on='id', how='left')
persons_geov = persons_geov.merge(genders, on='id', how='left')
persons_geov = persons_geov.merge(births, on='id', how='left')
persons_geov = persons_geov.merge(deaths, on='id', how='left')
persons_geov['dataset'] = 'geov'
persons_geov.dropna(subset=['name', 'gender', 'birth_year', 'death_year'], how='all', inplace=True)
persons_geov.drop_duplicates(inplace=True)

tk.infos(persons_geov, random=True)

Shape:  (147644, 6)


,id,name,gender,birth_year,death_year,dataset
96184,3463970,gheorghe vasile,Male,NaN,NaN,geov
44141,1344148,weil johann georg,NaN,NaN,NaN,geov
24593,337137,häfeli maria,NaN,1822.0,NaN,geov
45372,1345221,lemann barbara,NaN,NaN,NaN,geov
68413,1751694,domenico di martino calotti (dalla diocesi di ...,Male,1553.0,NaN,geov


In [18]:
persons_geov.to_csv('../../data/persons-geov.csv', index=False, sep=";", quoting=2)

# Merge BHP persons and Geovistory persons

In [19]:
persons = pd.concat([persons_bhp, persons_geov])
tk.set_types(persons, {'name':'string', 'gender':'string','birth_year': 'int','death_year': 'int'})

persons.reset_index(inplace=True, drop=True)
persons.reset_index(inplace=True)
persons.rename(columns={'index':'unique_id'}, inplace=True)

persons.to_csv('../../data/persons-bhp-geov.csv', index=False, sep=";", quoting=2)
